In [1]:
import os
import folium
from IPython.display import display, display_html, IFrame

### Funcs

In [2]:
def visualize_coordinates_in_notebook(point_list):
    """
    Visualize a list of coordinates on a map directly in a Jupyter Notebook.

    :param point_list: List of dictionaries containing latitude and longitude.
    """
    # Create a folium map centered around the average latitude and longitude
    center_lat = sum(point["latitude"] for point in point_list) / len(point_list)
    center_lon = sum(point["longitude"] for point in point_list) / len(point_list)
    map_ = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    plot_polygon = folium.Polygon(
        locations=[(point["latitude"], point["longitude"]) for point in point_list],
        color="#2196F3",
        weight=2,
        fill=True,
        fill_color="#2196F3",
        fill_opacity=0.4,
    )

    plot_polygon.add_to(map_)

    # Add points to the map
    for point in point_list:
        folium.Marker(
            location=[point["latitude"], point["longitude"]],
            popup=f"Lat: {point['latitude']}, Lon: {point['longitude']}",
        ).add_to(map_)

    # Save the map to a temporary HTML file
    map_file = "map.html"
    map_.save(map_file)

    full_path = os.path.abspath(map_file)
    print(full_path)

In [4]:
def rearrange_coordinates(point_list, sort_by="latitude"):
    """
    Rearrange a list of coordinates in order based on the given sorting criterion.

    :param point_list: List of dictionaries containing latitude and longitude.
    :param sort_by: The key to sort by ("latitude", "longitude", or "both").
    :return: Sorted list of coordinates.
    """
    if sort_by == "latitude":
        return sorted(point_list, key=lambda point: point["latitude"])
    elif sort_by == "longitude":
        return sorted(point_list, key=lambda point: point["longitude"])
    elif sort_by == "both":
        return sorted(
            point_list, key=lambda point: (point["latitude"], point["longitude"])
        )
    else:
        raise ValueError(
            "Invalid sort_by value. Use 'latitude', 'longitude', or 'both'."
        )

In [6]:
import math


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth.
    :param lat1, lon1: Latitude and Longitude of point 1 in decimal degrees.
    :param lat2, lon2: Latitude and Longitude of point 2 in decimal degrees.
    :return: Distance in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def sort_by_distance(point_list):
    """
    Sort points by distance from the first point in the list.
    :param point_list: List of dictionaries containing latitude and longitude.
    :return: List of points sorted by distance from the first point.
    """
    if not point_list:
        return []

    # Get the reference point (the first point in the list)
    ref_point = point_list[0]
    ref_lat, ref_lon = ref_point["latitude"], ref_point["longitude"]

    # Calculate distance from the reference point and sort the list
    sorted_list = sorted(
        point_list,
        key=lambda point: haversine_distance(
            ref_lat, ref_lon, point["latitude"], point["longitude"]
        ),
    )
    return sorted_list

In [9]:
import math


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth.
    :param lat1, lon1: Latitude and Longitude of point 1 in decimal degrees.
    :param lat2, lon2: Latitude and Longitude of point 2 in decimal degrees.
    :return: Distance in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def order_points_by_proximity(point_list):
    """
    Order points by proximity chaining, starting with the first point.
    :param point_list: List of dictionaries with latitude and longitude.
    :return: List of points in the correct order based on proximity.
    """
    if not point_list:
        return []

    ordered_points = [point_list[0]]  # Start with the first point
    remaining_points = point_list[1:]  # Remaining points to visit

    while remaining_points:
        # Get the last point in the ordered list
        current_point = ordered_points[-1]
        current_lat, current_lon = current_point["latitude"], current_point["longitude"]

        # Find the nearest point among the remaining
        nearest_point = min(
            remaining_points,
            key=lambda point: haversine_distance(
                current_lat, current_lon, point["latitude"], point["longitude"]
            ),
        )

        # Add the nearest point to the ordered list and remove it from remaining points
        ordered_points.append(nearest_point)
        remaining_points.remove(nearest_point)

    return ordered_points

In [10]:
def order_points_by_bearing(point_list):
    """
    Order points based on clockwise bearing from the first point.
    """
    ref_point = point_list[0]
    ref_lat, ref_lon = ref_point["latitude"], ref_point["longitude"]

    def bearing(point):
        lat, lon = point["latitude"], point["longitude"]
        angle = math.atan2(lon - ref_lon, lat - ref_lat)
        return (math.degrees(angle) + 360) % 360  # Normalize to [0, 360)

    return sorted(point_list, key=bearing)

In [11]:
import rtree
from shapely.geometry import Point

def reorder_points(points):
    """Reorders a list of points based on spatial proximity.

    Args:
        points: A list of dictionaries, each containing 'latitude' and 'longitude' keys.

    Returns:
        A list of reordered points.
    """
    # Create an R-tree index
    index = rtree.index.Index()
    for i, point in enumerate(points):
        index.insert(i, (point['longitude'], point['latitude'], point['longitude'], point['latitude']))

    # Find the nearest neighbor for each point
    reordered_points = []
    visited = set()
    current_index = 0  # Start with the first point

    while len(reordered_points) < len(points):
        reordered_points.append(points[current_index])
        visited.add(current_index)

        # Find the nearest neighbor that has not been visited
        neighbors = index.nearest(
            (points[current_index]['longitude'], points[current_index]['latitude']) * 2, 
            len(points)
        )
        for neighbor in neighbors:
            if neighbor not in visited:
                current_index = neighbor
                break

    return reordered_points
 

In [12]:
def corr_arrange_points(points):
    """
    Rearrange points to form a non-intersecting polygon using Graham Scan algorithm.
    
    Args:
        points (list): List of dictionaries containing latitude and longitude coordinates
        
    Returns:
        list: Rearranged points forming a non-intersecting polygon
    """
    def find_bottom_point(points):
        # Find the point with the lowest y-coordinate (latitude)
        return min(points, key=lambda p: (p['latitude'], p['longitude']))
    
    def calculate_angle(p1, p2):
        # Calculate the angle between two points relative to the horizontal
        import math
        dx = p2['longitude'] - p1['longitude']
        dy = p2['latitude'] - p1['latitude']
        return math.atan2(dy, dx)
    
    def orientation(p1, p2, p3):
        # Calculate the orientation of three points
        # Returns: 0 --> collinear, 1 --> clockwise, 2 --> counterclockwise
        val = ((p2['latitude'] - p1['latitude']) * (p3['longitude'] - p2['longitude']) -
               (p2['longitude'] - p1['longitude']) * (p3['latitude'] - p2['latitude']))
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    if len(points) < 3:
        return points

    # Find the bottommost point
    bottom_point = find_bottom_point(points)
    
    # Sort points based on polar angle with respect to the bottom point
    sorted_points = sorted(
        [p for p in points if p != bottom_point],
        key=lambda p: (calculate_angle(bottom_point, p), 
                      (p['longitude'] - bottom_point['longitude'])**2 + 
                      (p['latitude'] - bottom_point['latitude'])**2)
    )
    
    # Initialize the stack with the first three points
    stack = [bottom_point, sorted_points[0]]
    
    # Process remaining points
    for i in range(1, len(sorted_points)):
        while len(stack) > 1 and orientation(stack[-2], stack[-1], sorted_points[i]) != 2:
            stack.pop()
        stack.append(sorted_points[i])
    
    return stack

### Data

In [ ]:
data = {
  "plot_info": {
    "plot_number": "15B",
    "area": 1.78,
    "metric": "Acres",
    "locality": "KNUST",
    "district": "OFORIKROM",
    "region": "ASHANTI",
    "owners": [
      "TRANSPORT RESEARCH & EDUCATION CENTRE",
      "KUMASI (TRECK)"
    ],
    "date": "02/02/2022",
    "scale": "1:2500",
    "other_location_details": "Research Hills",
    "surveyors_name": "DR. A. ARKO-ADJEI",
    "surveyors_location": "P. O. BOX UP 1703 KNUST-KUMASI",
    "surveyors_reg_number": "316",
    "regional_number": None,
    "reference_number": None
  },
  "survey_points": [
    {
      "point_name": "KNUST.TREK.10/2021/1",
      "original_coords": {
        "x": 724125.686,
        "y": 695158.748,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.6655264110112284,
        "longitude": -1.5645176483991616,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    },
    {
      "point_name": "KNUST.TREK.10/2021/2",
      "original_coords": {
        "x": 724103.844,
        "y": 695149.339,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.665466169857804,
        "longitude": -1.5645435212323173,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    },
    {
      "point_name": "KNUST.TREK.10/2021/3",
      "original_coords": {
        "x": 724089.96,
        "y": 695129.526,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.665427833607018,
        "longitude": -1.5645981041468533,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    },
    {
      "point_name": "KNUST.TREK.10/2021/4",
      "original_coords": {
        "x": 724057.009,
        "y": 694842.085,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.665336091508574,
        "longitude": -1.565390507351992,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    },
    {
      "point_name": "KNUST.TREK.10/2021/5",
      "original_coords": {
        "x": 724197.311,
        "y": 694820.23,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.665722790800521,
        "longitude": -1.565451206986715,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    },
    {
      "point_name": "KNUST.CEPB.10/2021/2",
      "original_coords": {
        "x": 724330.685,
        "y": 695135.853,
        "ref_point": False
      },
      "converted_coords": {
        "latitude": 6.666091456391503,
        "longitude": -1.5645814187533298,
        "ref_point": False
      },
      "next_point": {
        "name": None,
        "bearing": None,
        "bearing_decimal": None,
        "distance": [
          None
        ]
      }
    }
  ],
  "boundary_points": [
    {
      "point": "Boundary_1",
      "northing": 694500,
      "easting": 723500,
      "latitude": 6.583939713344206,
      "longitude": -1.4862971406114294
    },
    {
      "point": "Boundary_2",
      "northing": 695500,
      "easting": 724500,
      "latitude": 6.58669911679171,
      "longitude": -1.4835431116150368
    }
  ],
  "point_list": [
    {
      "latitude": 6.665336091508574,
      "longitude": -1.565390507351992,
      "ref_point": False
    },
    {
      "latitude": 6.665427833607018,
      "longitude": -1.5645981041468533,
      "ref_point": False
    },
    {
      "latitude": 6.665466169857804,
      "longitude": -1.5645435212323173,
      "ref_point": False
    },
    {
      "latitude": 6.6655264110112284,
      "longitude": -1.5645176483991616,
      "ref_point": False
    },
    {
      "latitude": 6.666091456391503,
      "longitude": -1.5645814187533298,
      "ref_point": False
    },
    {
      "latitude": 6.665722790800521,
      "longitude": -1.565451206986715,
      "ref_point": False
    }
  ]
}

point_list = data["point_list"]

6

### Run

In [14]:
# Visualize sorted by latitude
sorted_by_latitude = sorted(point_list, key=lambda x: x["latitude"])
visualize_coordinates_in_notebook(sorted_by_latitude)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [15]:
# Visualize sorted by latitude
sorted_by_longitude = rearrange_coordinates(point_list, sort_by="longitude")
visualize_coordinates_in_notebook(sorted_by_longitude)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [16]:
# Sort by both latitude and longitude
sorted_by_both = rearrange_coordinates(point_list, sort_by="both")
visualize_coordinates_in_notebook(sorted_by_both)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [17]:
sorted_by_distance = sort_by_distance(point_list)
visualize_coordinates_in_notebook(point_list)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [18]:
# Get ordered points
ordered_points = order_points_by_proximity(point_list)
# Print the results
visualize_coordinates_in_notebook(ordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [21]:
# Get ordered points
ordered_points = order_points_by_bearing(point_list)
# Print the results
visualize_coordinates_in_notebook(ordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [20]:

reordered_points = reorder_points(point_list)
# Print the results
visualize_coordinates_in_notebook(reordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [22]:
reordered_points = corr_arrange_points(point_list)
# Print the results
visualize_coordinates_in_notebook(reordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


### Spot Ref point

In [34]:
def find_reference_point(points):
    """
    Identifies the reference point in a set of coordinates by finding the point
    that's furthest from the centroid of all points.
    
    Args:
        points (list): List of dictionaries containing latitude, longitude, and ref_point
        
    Returns:
        dict: The identified reference point and its index
    """
    def calculate_distance(p1, p2):
        """Calculate the Euclidean distance between two points."""
        return ((p1['latitude'] - p2['latitude'])**2 + 
                (p1['longitude'] - p2['longitude'])**2)**0.5
    
    def calculate_centroid(points):
        """Calculate the centroid of all points."""
        if not points:
            return None
        
        lat_sum = sum(p['latitude'] for p in points)
        lon_sum = sum(p['longitude'] for p in points)
        count = len(points)
        
        return {
            'latitude': lat_sum / count,
            'longitude': lon_sum / count,
            'ref_point': False
        }
    
    def find_outlier_score(points):
        """
        Calculate an outlier score for each point based on its average distance
        to all other points.
        """
        scores = []
        for i, point in enumerate(points):
            total_distance = 0
            for j, other_point in enumerate(points):
                if i != j:
                    total_distance += calculate_distance(point, other_point)
            avg_distance = total_distance / (len(points) - 1)
            scores.append((avg_distance, i))
        return scores

    # Calculate outlier scores
    outlier_scores = find_outlier_score(points)
    
    # Find the point with the highest outlier score
    max_score, max_index = max(outlier_scores)
    return {
        'reference_point': points[max_index],
        'index': max_index
    }


In [35]:
# Find the reference point
result = find_reference_point(point_list)

# Print results
print(f"Reference point found at index: {result['index']}")
print(f"Coordinates: Lat {result['reference_point']['latitude']}, Lon {result['reference_point']['longitude']}")

Reference point found at index: 3
Coordinates: Lat 5.777552603198304, Lon -0.097168428017529
